In [1]:
import pandas as pd

true = pd.read_csv('True.csv', on_bad_lines='skip')

In [2]:
true.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3703 entries, 0 to 3702
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    3703 non-null   object
 1   text     3703 non-null   object
 2   subject  3702 non-null   object
 3   date     3702 non-null   object
dtypes: object(4)
memory usage: 115.8+ KB


In [3]:
true = true[:100]
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
all_text = '\n'.join(true['text'])

In [5]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer

In [6]:
tokenizer = Tokenizer()

In [7]:
tokenizer.fit_on_texts([all_text])

In [8]:
len(tokenizer.word_index)

6042

In [9]:
input_sequences = []
for sentence in all_text.split('\n'):
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[:i+1])

In [11]:
max_len = max([len(x) for x in input_sequences])

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='pre')

In [13]:
padded_input_sequences

array([[   0,    0,    0, ...,    0,   52,   45],
       [   0,    0,    0, ...,   52,   45,    1],
       [   0,    0,    0, ...,   45,    1, 1341],
       ...,
       [   0,    0,    0, ...,  182,   10,    5],
       [   0,    0,    0, ...,   10,    5,   34],
       [   0,    0,    0, ...,    5,   34,  359]], dtype=int32)

In [14]:
X = padded_input_sequences[:, :-1]

In [15]:
y = padded_input_sequences[:,-1]

In [16]:
X.shape

(43034, 1400)

In [17]:
y.shape

(43034,)

In [18]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=y.max() + 1)

In [19]:
y.shape

(43034, 6043)

In [20]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [22]:
model = Sequential()
model.add(Embedding(6043, 100, input_length=1400))
model.add(LSTM(150))
model.add(Dense(6043, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [25]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [26]:
model.fit(X, y, epochs=50, callbacks=[early_stop])

Epoch 1/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 77s 53ms/step - accuracy: 0.0569 - loss: 7.2826
Epoch 2/50


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


1345/1345 ━━━━━━━━━━━━━━━━━━━━ 82s 55ms/step - accuracy: 0.0890 - loss: 6.4308
Epoch 3/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 74s 55ms/step - accuracy: 0.1190 - loss: 5.8996
Epoch 4/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 82s 55ms/step - accuracy: 0.1461 - loss: 5.3950
Epoch 5/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 82s 55ms/step - accuracy: 0.1827 - loss: 4.8582
Epoch 6/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 82s 55ms/step - accuracy: 0.2195 - loss: 4.3749
Epoch 7/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 82s 55ms/step - accuracy: 0.2639 - loss: 3.9213
Epoch 8/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 74s 55ms/step - accuracy: 0.3184 - loss: 3.5259
Epoch 9/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 74s 55ms/step - accuracy: 0.3864 - loss: 3.1268
Epoch 10/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 82s 55ms/step - accuracy: 0.4509 - loss: 2.7444
Epoch 11/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 83s 56ms/step - accuracy: 0.5086 - loss: 2.4383
Epoch 12/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 81s 55ms/step - accuracy: 0.5641 - loss: 2.1403
Epoch 13/50
1345/1

In [43]:
import numpy as np

text = "total duration"

for i in range(10):

  #tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]

  #padding
  padded_token_text = pad_sequences([token_text], maxlen=max_len, padding='pre')

  #predict
  word_num = np.argmax(model.predict(padded_token_text))

  for word, index in tokenizer.word_index.items():
    if index == word_num:
      text = text + " " + word
      print(text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
total duration of
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
total duration of the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
total duration of the u
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
total duration of the u s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
total duration of the u s house
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
total duration of the u s house of
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
total duration of the u s house of representatives
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
total duration of the u s house of representatives on
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
total duration of the u s house of representatives on thursday
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
total duration of the u s house of representatives on thursday took


In [44]:
model.save('word_predictor.keras')

In [45]:
import pickle

# Save
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [46]:
from tensorflow.keras.models import load_model

In [47]:
model = load_model('word_predictor.keras')

In [48]:
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [49]:
import numpy as np

text = "America is"

for i in range(100):

  #tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]

  #padding
  padded_token_text = pad_sequences([token_text], maxlen=max_len, padding='pre')

  #predict
  word_num = np.argmax(model.predict(padded_token_text))

  for word, index in tokenizer.word_index.items():
    if index == word_num:
      text = text + " " + word
      print(text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
America is to
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
America is to make
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
America is to make conservative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
America is to make conservative members
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
America is to make conservative members of
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
America is to make conservative members of u
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
America is to make conservative members of u s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
America is to make conservative members of u s district
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
America is to make conservative members of u s district judge
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
America is to make conservative members of u s district judge george
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
America is to make conservative members of u s district judge george ruled
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
America is to make conservative members o